In [ ]:
#Track 1: Customer Segmentation

# Can you identify distinct user segments based on their reviews?
# Ans: yes, cluster 0, 1, 2, 3.
# Cluster Summary:
# Cluster 0: 16007 users (35.56%)
# Cluster 1: 9901 users (21.99%)
# Cluster 2: 15049 users (33.43%)
# Cluster 3: 4062 users (9.02%)

# What are the key characteristics of each segment?
# Ans: cluster 0 are 'enthusiastic users' who are enthusiastic about writing reviews and have many fans.
# Ans: cluster 3 are 'invisible users' who normally won't give reviews, tend to give lower scores and don't have many fans.
# Ans: cluster 1 are 'regular, picky users' who are not very active nor invisible, and tend to give relatively low score.
# Ans: cluster 2 are 'regular, undemanding users' who are not very active nor invisible, and tend to give relatively high score.
# Cluster Means:
# Cluster                      0       1       2       3  Overall
# avg_review_length       585.96  646.60  405.70  621.28   542.23
# review_frequency         11.04    5.16    5.42    0.94     6.96
# category_diversity        6.04    5.38    5.56    4.96     5.64
# avg_rating_deviation      0.48   -1.35    0.76   -1.56    -0.02
# engagement_score       1030.35  217.44  196.60   12.77   481.05
# tenure_days            4965.37 3859.08 3148.60 3136.83  3949.77
# average_stars_usercsv     3.91    3.35    4.36    1.67     3.73
# compliment_hot_usercsv   23.16    3.52    2.58    0.01     9.88
# fans_usercsv             15.43    3.23    3.33    0.12     7.32
# review_count_usercsv    155.12   56.77   46.70    8.29    84.00

# How can marketing strategies be tailored to each segment to promote businesses and increase their customer base?
# Ans: cluster 0, compared to other clusters, care more about bike parking and reservation, so we can promote those when marketing.
# Ans: cluster 1, compared to other clusters, care more about bike RestaurantsTableService and AcceptsCreditCards, so we can promote those when marketing.
# Ans: cluster 2, compared to other clusters, care more about caters, AcceptsCreditCards and OutdoorSeating, so we can promote those when marketing.
# Ans: cluster 3, compared to other clusters, care more about GoodForKids, HasTV and Delivery, so we can promote those when marketing.
# Business Attribute Ratios by Cluster (% of 'TRUE' values):
# Cluster                                                 0       1       2       3 Overall
# attributes.GoodForKids_businesscsv                 77.92%  82.15%  83.43%  84.31%  80.99%
# attributes.DogsAllowed_businesscsv                 25.74%  24.93%  27.32%  23.22%  26.01%
# attributes.RestaurantsDelivery_businesscsv         62.92%  66.75%  69.26%  72.00%  66.43%
# attributes.Caters_businesscsv                      54.44%  55.44%  59.07%  58.14%  56.40%
# attributes.RestaurantsTableService_businesscsv     75.22%  76.95%  75.99%  74.52%  75.82%
# attributes.OutdoorSeating_businesscsv              64.49%  61.94%  67.34%  58.80%  64.47%
# attributes.HasTV_businesscsv                       70.29%  75.84%  73.30%  80.19%  73.11%
# attributes.BusinessAcceptsCreditCards_businesscsv  96.02%  97.21%  97.21%  96.99%  96.73%
# attributes.BikeParking_businesscsv                 82.85%  80.21%  81.75%  75.96%  81.46%
# attributes.RestaurantsReservations_businesscsv     47.68%  46.43%  45.61%  41.22%  46.32%
# attributes.DriveThru_businesscsv                   24.69%  30.33%  24.01%  43.93%  27.48%
# attributes.Open24Hours_businesscsv                 26.53%  10.00%  41.18%  28.57%  27.71%

In [ ]:
# import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
##########################################################
# Please update the "File paths" part below before running
##########################################################
# File paths
business_path = '/Users/sz229/Desktop/EAS Project/Data/business.csv'
review_path = '/Users/sz229/Desktop/EAS Project/Data/review.csv'
user_path = '/Users/sz229/Desktop/EAS Project/Data/user.csv'

# Load the datasets
business_df = pd.read_csv(business_path)
review_df = pd.read_csv(review_path)
user_df = pd.read_csv(user_path)


# PART 1: Cleaning data

In [ ]:
# Add suffixes to all columns before merging to make sure we track the right attributes
business_df.columns = [f"{col}_businesscsv" for col in business_df.columns]
review_df.columns = [f"{col}_reviewcsv" for col in review_df.columns]
user_df.columns = [f"{col}_usercsv" for col in user_df.columns]

In [ ]:
# Rename the 'business_id' and 'user_id' columns back to their original names for merging
business_df = business_df.rename(columns={'business_id_businesscsv': 'business_id'})
review_df = review_df.rename(columns={'business_id_reviewcsv': 'business_id', 'user_id_reviewcsv': 'user_id'})
user_df = user_df.rename(columns={'user_id_usercsv': 'user_id'})

In [ ]:
# Merge datasets
merged_df = review_df.merge(business_df, on='business_id')
merged_df = merged_df.merge(user_df, on='user_id')

In [ ]:
# Function to display basic information about a dataframe
def display_df_info(df, name):
    print(f"\n--- {name} Dataset ---")
    print(f"Shape: {df.shape}")
    print("\nColumn names:")
    print(df.columns.tolist())
    print("\nData types:")
    print(df.dtypes)
    print("\nMissing values:")
    print(df.isnull().sum())
    print("\nSample data:")
    print(df.head())

In [ ]:
# Display information for the merged dataset
display_df_info(merged_df, "Merged")

In [ ]:
# Function to plot distribution of a numerical column
def plot_distribution(df, column, title):
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], kde=True)
    plt.title(title)
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.show()

In [ ]:
# Plot distributions of some variables
plot_distribution(merged_df, 'stars_reviewcsv', 'Distribution of Review Stars')
plot_distribution(merged_df, 'average_stars_usercsv', 'Distribution of User Average Stars')

In [ ]:
# Select relevant numerical columns for correlation analysis
numerical_columns = ['stars_reviewcsv', 'useful_reviewcsv', 'funny_reviewcsv', 'cool_reviewcsv',
                     'review_count_usercsv', 'useful_usercsv', 'funny_usercsv', 'cool_usercsv',
                     'fans_usercsv', 'average_stars_usercsv', 'compliment_hot_usercsv']

In [ ]:
# Calculate the correlation matrix round 2 decimal
correlation_matrix = merged_df[numerical_columns].corr()
correlation_matrix_rounded = correlation_matrix.round(2)

# Print the correlation matrix
print("\nCorrelation Matrix:")
print(correlation_matrix_rounded)

In [ ]:
# Find the highest correlations
correlations = correlation_matrix.unstack()
correlations = correlations[correlations < 1.0]  # remove self-corr
high_correlations = correlations.sort_values(ascending=False).head(10)

print("\nTop 10 Highest Correlations:")
print(high_correlations)

In [ ]:
# Print summary statistics
print("\nSummary Statistics:")
print(merged_df[numerical_columns].describe())

In [ ]:
# Identify and print top categories
top_categories = merged_df['categories_businesscsv'].str.split(', ', expand=True).stack().value_counts().head(10)
print("\nTop 10 Business Categories:")
print(top_categories)

# PART 2: Add more features

In [ ]:
# 1. Average review length
merged_df['review_length_reviewcsv'] = merged_df['text_reviewcsv'].str.len()
merged_df['avg_review_length'] = merged_df.groupby('user_id')['review_length_reviewcsv'].transform('mean')

print(merged_df['avg_review_length'])

In [ ]:
# 2. Frequency of reviews (reviews per year)
merged_df['yelping_since_usercsv'] = pd.to_datetime(merged_df['yelping_since_usercsv'])
current_date = pd.Timestamp.now()
merged_df['years_on_yelp'] = (current_date - merged_df['yelping_since_usercsv']).dt.total_seconds() / (365.25 * 24 * 60 * 60)
merged_df['review_frequency'] = merged_df['review_count_usercsv'] / merged_df['years_on_yelp']

print(merged_df['review_frequency'])

In [ ]:
# 3. Diversity of businesses reviewed (unique categories)
# First, we need to explode the categories and count unique ones per user
categories_df = merged_df.groupby('user_id')['categories_businesscsv'].apply(
    lambda x: pd.Series(x.str.split(', ', expand=True).values.ravel()).dropna().unique()
)
merged_df['category_diversity'] = merged_df['user_id'].map(categories_df.str.len())

print('category_diversity', merged_df['category_diversity'])

In [ ]:
# 4. Average rating deviation (how much a user's ratings deviate from the business average)
merged_df['rating_deviation'] = merged_df['stars_reviewcsv'] - merged_df['stars_businesscsv']
merged_df['avg_rating_deviation'] = merged_df.groupby('user_id')['rating_deviation'].transform('mean')

print('avg_rating_deviation', merged_df['avg_rating_deviation'])

In [ ]:
# 5. Engagement score (combination of useful, funny, and cool votes received)
merged_df['engagement_score'] = (merged_df['useful_usercsv'] +
                                 merged_df['funny_usercsv'] +
                                 merged_df['cool_usercsv'])

print('engagement_score', merged_df['engagement_score'])

In [ ]:
# 6. Tenure as days (how long a user has been with Yelp)
merged_df['yelping_since_usercsv'] = pd.to_datetime(merged_df['yelping_since_usercsv'])
merged_df['tenure_days'] = (pd.Timestamp.now() - merged_df['yelping_since_usercsv']).dt.days

print('tenure_days', merged_df['tenure_days'])

# PART 3: Feature preparation

In [ ]:
# Select features for clustering
clustering_features = [
    'avg_review_length',
    'review_frequency',
    'category_diversity',
    'avg_rating_deviation',
    'engagement_score',
    'tenure_days',

    'average_stars_usercsv',
    'compliment_hot_usercsv',
    'fans_usercsv',
    #'friends_usercsv' # see how number of friends can influence user activity
    'review_count_usercsv',
]

In [ ]:
# Create a new dataframe with one row per user, using the mean of their features
user_features_df = merged_df.groupby('user_id')[clustering_features].mean().reset_index()

In [ ]:
# Function to normalize a column using Min-Max scaling
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)

In [ ]:
# Normalize features
user_features_normalized = user_features_df.copy()
for feature in clustering_features:
    user_features_normalized[f'{feature}_normalized'] = normalize_column(user_features_normalized[feature])

In [ ]:
# Keep only normalized features and user_id
columns_to_keep = [f'{feature}_normalized' for feature in clustering_features] + ['user_id']
user_features_normalized = user_features_normalized[columns_to_keep]

In [ ]:
# Display info about the new features
print("#################################")
print(user_features_normalized.head(10))

# PART 4: Segmentation

In [ ]:
# Load the normalized user features
user_features = user_features_normalized

In [ ]:
# Select the features for clustering
clustering_features = [
    'avg_review_length_normalized',
    'review_frequency_normalized',
    'category_diversity_normalized',
    'avg_rating_deviation_normalized',
    'engagement_score_normalized',
    'tenure_days_normalized',
    'average_stars_usercsv_normalized',
    'compliment_hot_usercsv_normalized',
    'fans_usercsv_normalized',
    'review_count_usercsv_normalized',
]

In [ ]:
# Extract features for clustering
X = user_features[clustering_features].values

In [ ]:
# Implement elbow method
def elbow_method(X, max_k):
    inertias = []
    for k in range(1, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    return inertias

In [ ]:
# Run elbow method
max_k = 10
inertias = elbow_method(X, max_k)

In [ ]:
# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, max_k + 1), inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()
plt.close()

In [ ]:
# Determine optimal k
optimal_k = 4

# Perform final clustering with optimal k
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
labels = kmeans.fit_predict(X)

# Add cluster labels to the user features dataframe
user_features['Cluster'] = labels

In [ ]:
# Print summary of clusters
print("\nCluster Summary:")
for i in range(optimal_k):
    cluster_size = sum(labels == i)
    print(f"Cluster {i}: {cluster_size} users ({cluster_size/len(labels)*100:.2f}%)")

In [ ]:
# Calculate and print cluster centroids
cluster_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=clustering_features)
print("\nCluster Centroids:")
print(cluster_centroids)

**PCA**

In [ ]:
#########PCA#########
# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
# Visualize clusters using PCA
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.scatter(pca.transform(kmeans.cluster_centers_)[:, 0],
            pca.transform(kmeans.cluster_centers_)[:, 1],
            marker='x', s=200, linewidths=3, color='red', label='Centroids')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Customer Segments (PCA)')
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.legend()
plt.show()
plt.close()

# PART 5: Post-Segmentation Analytics for each cluster: characteristics

In [ ]:
# Merge the cluster labels to the original dataset using user_id
user_features_df = user_features_df.merge(user_features_normalized[['user_id', 'Cluster']],
                                          on='user_id',
                                          how='left',
                                          validate='one_to_one')

In [ ]:
# Check if any users are missing cluster labels
missing_clusters = user_features_df['Cluster'].isnull().sum()
if missing_clusters > 0:
    print(f"Warning: {missing_clusters} users are missing cluster labels.")
else:
    print("All users have been assigned cluster labels successfully.")

In [ ]:
# Display summary statistics for each cluster
# List of features we want to analyze
features = [
    'avg_review_length', 'review_frequency', 'category_diversity',
    'avg_rating_deviation', 'engagement_score', 'tenure_days',
    'average_stars_usercsv', 'compliment_hot_usercsv', 'fans_usercsv',
    'review_count_usercsv'
]

In [ ]:
# Calculate means for each cluster
cluster_means = user_features_df.groupby('Cluster')[features].mean()

# Calculate overall means
overall_means = user_features_df[features].mean()

# Transpose the DataFrame for a more compact display
cluster_means_t = cluster_means.T

# Add overall means as a column
cluster_means_t['Overall'] = overall_means

In [ ]:
# Format the output
pd.set_option('display.float_format', '{:.2f}'.format)

print("Cluster Means:")
print(cluster_means_t)

# PART 6: Post-Segmentation Analytics for each cluster: business categories

In [ ]:
# Merge the cluster labels back to merged_df
merged_df_with_clusters = merged_df.merge(user_features_df[['user_id', 'Cluster']], on='user_id', how='left')

In [ ]:
# Function to get top categories for a given cluster
def get_top_categories(df, cluster):
    cluster_data = df[df['Cluster'] == cluster]
    categories = cluster_data['categories_businesscsv'].str.split(', ', expand=True).stack()
    return categories.value_counts().head(10)

In [ ]:
# Get top 10 categories for each cluster
for cluster in merged_df_with_clusters['Cluster'].unique():
    print(f"\nTop 10 Business Categories for Cluster {cluster}:")
    top_categories = get_top_categories(merged_df_with_clusters, cluster)
    print(top_categories)
    print("=================================================")

# PART 7: Post-Segmentation Analytics for each cluster: business attributes

In [ ]:
# List of attributes we want to analyze
attributes = [
    'attributes.GoodForKids_businesscsv',
    'attributes.DogsAllowed_businesscsv',
    'attributes.RestaurantsDelivery_businesscsv',
    'attributes.Caters_businesscsv',
    'attributes.RestaurantsTableService_businesscsv',
    'attributes.OutdoorSeating_businesscsv',
    'attributes.HasTV_businesscsv',
    'attributes.BusinessAcceptsCreditCards_businesscsv',
    'attributes.BikeParking_businesscsv',
    'attributes.RestaurantsReservations_businesscsv'
    'attributes.DriveThru_businesscsv',
    'atrributes.Open24Hours_businesscsv'
]

In [ ]:
# Check data types and unique values
print("Data types:")
print(merged_df_with_clusters[attributes].dtypes)

In [ ]:
# Function to calculate the ratio of 'TRUE' values
def true_ratio(series):
    true_values = series.astype(str).str.lower() == 'true'
    total_values = series.notna()
    return true_values.sum() / total_values.sum() if total_values.sum() > 0 else np.nan

In [ ]:
# Calculate ratios for each attribute by cluster
attribute_ratios = merged_df_with_clusters.groupby('Cluster')[attributes].apply(lambda x: x.apply(true_ratio))

# Calculate overall ratios
overall_ratios = merged_df_with_clusters[attributes].apply(true_ratio)

# Add overall ratios as a new row instead of a column
attribute_ratios.loc['Overall'] = overall_ratios

In [ ]:
# Function to format percentages
def format_percentage(x):
    return f"{x:.2%}" if pd.notna(x) else "N/A"

In [ ]:
# Apply formatting
attribute_ratios_pct = attribute_ratios.apply(lambda col: col.map(format_percentage))

# Transpose for better readability
attribute_ratios_pct = attribute_ratios_pct.T

In [ ]:
# Display the results
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Autodetect display width
pd.set_option('display.max_rows', None)  # Show all rows

print("\nBusiness Attribute Ratios by Cluster (% of 'TRUE' values):")
print(attribute_ratios_pct)